# Part 1

In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [2]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, wikipedia_page, to handle the contents of the website
wikipedia_page=requests.get(wikipedia_link)

#Store the contents of the website under wikipedia_page
wikipedia_page = lh.fromstring(wikipedia_page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = wikipedia_page.xpath('//tr')

#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [3]:
tr_elements = wikipedia_page.xpath('//tr')

#Create empty list
col=[]

i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print(name)
    col.append((name,[]))

Postcode
Borough
Neighbourhood



In [4]:

col

[('Postcode', []), ('Borough', []), ('Neighbourhood\n', [])]

In [5]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [6]:
[len(C) for (title,C) in col]

[288, 288, 288]

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [8]:
df = df.replace(r'\n','', regex=True)
df.columns = ['Postcode', 'Borough', 'Neighbourhood']
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
#Removing Not assigned zip codes and fixing df index
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)
df.index = range(len(df))
df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [10]:
df.shape

(211, 3)

# Part 2

In [11]:
import geocoder
import pandas as pd
import numpy as np

In [12]:
# initialize your variable to None
lat_lng_coords = None

#Create extra columns
df['Latitude'] = pd.Series("", index=df.index)
df['Longitude'] = pd.Series("", index=df.index)

df.columns

Index(['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'], dtype='object')

In [ ]:
# loop until you get the coordinates
i = 0

sum_latitude = sum(df['Latitude'] == '')

while sum_latitude > 0:
    #print('Missing coordinates: '+sum_latitude) 
    if df['Latitude'][i] == '':
        try:
            g = geocoder.google('{}, Toronto, Ontario'.format(df['Neighbourhood'][i]))
            lat_lng_coords = g.latlng
            if g.latlng != None:
                df['Latitude'][i] = lat_lng_coords[0]
                df['Longitude'][i] = lat_lng_coords[1]
        except:
            break
    i = i+1
    sum_latitude = sum(df['Latitude'] == '')


df

In [13]:
df1=pd.read_csv("C:\\Users\\hp\\Downloads\\Geospatial_Coordinates.csv")
df1

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


# Part 3

In [14]:
from geopy.geocoders import Nominatim 

#Use geopy library to get the latitude and longitude values of Toronto
address = 'Toronto,ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of ',address,' are {}, {}.'.format(latitude, longitude))

D:\Python\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geograpical coordinate of  Toronto,ON  are 43.653963, -79.387207.


In [ ]:
import folium # map rendering library

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
            
map_toronto